# Деплой ML модели

И снова здравствуйте! Если вы читаете этот текст, значит вы все сделали верно.

Эта лаборатория посвящена развертыванию реальной модели машинного обучения и проверке ее в работе. Более конкретно, вы развернете модель компьютерного зрения, обученную обнаруживать объекты на изображениях. Развертывание модели - это один из последних шагов в жизненном цикле машинного обучения. В этом задании используется предварительно обученная модель под названием [`YOLOV3`](https://pjreddie.com/darknet/yolo /). Эта модель очень удобна по двум причинам: она работает очень быстро и точно.

Последовательность шагов / задач, которые необходимо выполнить в этой лаборатории, выглядит следующим образом:
1. Проверить датасет изображений, используемый для обнаружения объектов
2. Посмотреть саму модель
3. Развернуть модель с помощью [`fast API`](https://fastapi.triangolo.com /)

## Обнаружение объектов с помощью YOLOV3

### Проверка изображений

Давайте взглянем на изображения, которые будут переданы в модель YOLOV3. Это даст представление о том, какие типы объектов присутствуют на изображениях. Эти изображения являются частью датасета[`ImageNet`](http://www.image-net.org/index ).

In [2]:
from IPython.display import Image, display

In [3]:
# Примеры изображений
image_files = [
    'apple.jpg',
    'clock.jpg',
    'oranges.jpg',
    'car.jpg'
]

for image_file in image_files:
    print(f"\nDisplaying image: {image_file}")
    display(Image(filename=f"images/{image_file}"))


Displaying image: apple.jpg


FileNotFoundError: ignored

### Обзор модели

Теперь, когда у вас есть представление об изображениях и присутствующих на них объектах, давайте посмотрим, способна ли модель правильно их обнаруживать и классифицировать.

Для этого вы будете использовать [`cvlib`](https://www.cvlib.net /), которая представляет собой очень простую, но мощную библиотеку для обнаружения объектов, использующую [`OpenCV`](https://docs.opencv.org/4.5.1 /) и [`Tensorflow`](https://www.tensorflow.org /).

Более конкретно, вы будете использовать [`detect_common_objects`](https://docs.cvlib.net/object_detection /) функция, которая принимает изображение, отформатированное в виде [`numpy array`](https://numpy.org/doc/stable/reference/generated/numpy.array.html ) и возвращает:

- `bbox`: список содержащий координаты ограничивающей рамки для обнаруженных объектов. 

        Пример:
    
    ```python
        [[32, 76, 128, 192], [130, 83, 220, 185]]
    ```
    

- `label`: список меток распознанных объектов.
    
        Пример:
    ```python
        ['apple', 'apple']
    ```


- `conf`: Список коэффициентов достоверности.
        Пример:
        
    ```python
        [0.6187325716018677, 0.42835739254951477]
    ```
    
В следующем разделе вы наглядно увидите эти элементы в действии.

### Создание функции detect_and_draw_box

Перед использованием модели создайте каталог, в котором вы можете хранить полученные изображения:

In [ ]:
import os

dir_name = "images_with_boxes"
if not os.path.exists(dir_name):
    os.mkdir(dir_name)

Давайте определим функцию `detect_and_draw_box`, которая принимает в качестве входных аргументов: **filename** файла в вашей системе, **model** и **confidence level**. С помощью этих входных данных она обнаруживает общие объекты на изображении и сохраняет новое изображение, отображающее ограничительные рамки рядом с обнаруженным объектом.

Вы можете спросить себя, почему эта функция получает модель в качестве входного аргумента? Из каких моделей можно выбрать? Ответ заключается в том, что `detect_common_objects` по умолчанию использует модель `yolov3`. Однако существует другой доступный вариант, который намного меньше и требует меньше вычислительной мощности.

Это версия `yolov3-tiny`. Как следует из названия модели, эта модель предназначена для ограниченных сред, которые не могут хранить большие модели. При этом возникает естественный компромисс: результаты менее точны, чем у полной модели. Тем не менее, она по-прежнему работает довольно хорошо. В дальнейшем я рекомендую вам придерживаться этой модели, так как она намного меньше обычной "yolov3", а загрузка ее предварительно обученных весов занимает меньше времени.

Выходные данные модели представляют собой вектор вероятностей присутствия различных объектов на изображении. Последний входной аргумент, уровень достоверности, определяет пороговое значение, которое должна превысить вероятность, чтобы сообщить, что данный объект обнаружен на предоставленном изображении. По умолчанию `detect_common_objects` использует для этого значение 0,5.

In [ ]:
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox


def detect_and_draw_box(filename, model="yolov3-tiny", confidence=0.5):
    """Обнаруживает объекты на изображении и создает новое изображение с ограничивающими рамками.

    Аргументы:
        filename (str): Имя файла изображения.
        model (str): Либо "yolov3", либо "yolov3-tiny". По умолчанию используется значение "yolov3-tiny".
        confidence (float, optional): Желаемый уровень достоверности. Значение по умолчанию равно 0,5.
    """
    
    # Изображения хранятся в каталоге images/
    img_filepath = f'images/{filename}'
    
    # Считывание изображения в массив numpy
    img = cv2.imread(img_filepath)
    
    # Выполняем обнаружение объекта
    bbox, label, conf = cv.detect_common_objects(img, confidence=confidence, model=model)
    
    # Печать имени файла текущего изображения
    print(f"========================\nImage processed: {filename}\n")
    
    # Печать обнаруженных объектов с уровнем достоверности
    for l, c in zip(label, conf):
        print(f"Detected object: {l} with confidence level of {c}\n")
    
    # Создаем новое изображение, включающее ограничительные рамки
    output_image = draw_bbox(img, bbox, label, conf)
    
    # Сохраняем изображение в каталоге images_with_boxes
    cv2.imwrite(f'images_with_boxes/{filename}', output_image)
    
    # Покажем изображения с ограничивающими рамками
    display(Image(f'images_with_boxes/{filename}'))

Давайте попробуем.

In [ ]:
for image_file in image_files:
    detect_and_draw_box(image_file)

## Изменение уровня достоверности

Похоже, обнаружение объекта прошло довольно успешно. Давайте попробуем на более сложном изображении, содержащем несколько объектов:

In [ ]:
detect_and_draw_box("fruits.jpg")

Модель ошибочно классифицировала апельсин как яблоко. Это может показаться странным, поскольку раньше она могла обнаружить одно яблоко, из этого можно предположить, что модель имеет четкое представление о том, как выглядит яблоко.

Один из вариантов почему это произошло заключается в том, что модель действительно обнаружила другие фрукты, но с уровнем достоверности ниже 0,5. Давайте проверим, является ли это верной гипотезой:

In [ ]:
detect_and_draw_box("fruits.jpg", confidence=0.2)

Снижая уровень достоверности, модель успешно обнаруживает большинство фруктов. Однако, чтобы правильно обнаружить присутствующие объекты, нам пришлось установить очень низкий уровень достоверности. В общем, вы должны быть осторожны при уменьшении или увеличении таких параметров, так как их изменение может привести к нежелательным результатам.

Что касается этого конкретного примера, когда апельсин был ошибочно классифицирован как яблоко, он служит напоминанием о том, что эти модели не идеальны, и это следует учитывать при их использовании для производственных задач.

## Развертывание модели с использованием fast API


### Размещение вашей модели обнаружения объектов на сервере

Теперь, когда вы знаете, как работает модель, пришло время ее развернуть! Разве это не круто?))

Прежде чем перейти к развертыванию, давайте кратко рассмотрим некоторые важные концепции и то, как они переводятся в "fastAPI`. Также создадим каталог для хранения изображений, загружаемых на сервер.

In [ ]:
dir_name = "images_uploaded"
if not os.path.exists(dir_name):
    os.mkdir(dir_name)

### Некоторые пояснения к концепции

#### Модель клиент-Сервер

Когда говорят о **деплое**, обычно подразумевается размещение всего программного обеспечения, необходимого для прогнозирования, на `сервере`. Делая это, "клиент" может взаимодействовать с моделью, отправляя `запросы` на сервер.

Важная вещь, на которой вам нужно сосредоточиться, заключается в том, что модель машинного обучения находится на сервере, ожидающем, пока клиенты отправят запросы на прогнозирование. Клиент должен предоставить информацию, необходимую модели для того, чтобы сделать прогноз. Имейте в виду, что обычно в одном запросе группируется много прогнозов. Сервер будет использовать предоставленную информацию для возврата прогнозов клиенту, который затем сможет использовать их.

Давайте начнем с создания экземпляра класса `Fast API`:

```python
app = FastAPI()
```

Следующим шагом будет использование этого экземпляра для создания конечных точек, которые будут обрабатывать логику прогнозирования (подробнее об этом далее). Как только весь код будет на месте, для запуска сервера вам нужно только использовать команду:

```python
uvicorn.run(app)
```

Ваш API создан с использованием fast API, но обслуживается он с помощью [`unicorn`](https://www.unicorne.org /), который представляет собой действительно быструю реализацию асинхронного интерфейса шлюза сервера (ASGI). Обе технологии тесно взаимосвязаны, и вам не нужно разбираться в деталях реализации.

#### Endpoints

Вы можете разместить несколько моделей машинного обучения на одном сервере. Чтобы это сработало, вы можете назначить каждой модели другую "конечную точку", чтобы всегда знать, какая модель используется. Конечная точка представлена шаблоном в `URL`. Например, если у вас есть веб-сайт под названием `myawesomemodel.com ` у вас могут быть три разные модели в следующих конечных точках:

- `myawesomemodel.com/count-cars/`
- `myawesomemodel.com/count-apples/`
- `myawesomemodel.com/count-plants/`

Каждая модель будет делать то, что предполагает шаблон названия.

В fastAPI вы определяете конечную точку, создавая функцию, которая будет обрабатывать всю логику для этой конечной точки и [декорировать](https://www.python.org/dev/peps/pep-0318 /) его с функцией, которая содержит информацию о разрешенном методе HTTP (подробнее об этом далее) и шаблоне в URL, который он будет использовать.

В следующем примере показано, как разрешить HTTP-запрос GET для конечной точки "/my-endpoint":

```python
@app.get("/my-endpoint")
def handle_endpoint():
    ...
    ...
```


#### HTTP запрос

Клиент и сервер взаимодействуют друг с другом по протоколу, называемому `HTTP`. Ключевая концепция здесь заключается в том, что при этом общении между клиентом и сервером используются некоторые глаголы для обозначения общих действий. Два очень распространенных глагола - это:

- `GET` -> Получает информацию с сервера.
- `POST` -> Отправляет серверу информацию, которую он использует для ответа.

Если ваш клиент выполняет "GET-запрос" к конечной точке сервера, вы получите некоторую информацию от этой конечной точки без необходимости предоставления дополнительной информации. В случае "POST-запроса" вы явно сообщаете серверу, что предоставите ему некоторую информацию, которая должна быть обработана каким-либо образом.

```python
@app.post("/my-other-endpoint")
def handle_other_endpoint(param1: int, param2: str):
    ...
    ...

```

Для POST запросов функция обработчика содержит параметры. В отличие от GET, запросы POST ожидают, что клиент предоставит серверу некоторую информацию. В этом случае мы указали два параметра: целое число и строку.


### ### Почему fastAPI?

С помощью fastAPI вы можете очень легко создавать веб-серверы для размещения ваших моделей. Кроме того, эта платформа чрезвычайно быстра и **имеет встроенный клиент, который можно использовать для взаимодействия с сервером**. Чтобы использовать его, вам нужно будет посетить конечную точку "/docs", в данном случае это означает посещение http://localhost:8000/docs . Разве это не удобно?

Хватит болтовни, погнали!

In [ ]:
import io
import uvicorn
import numpy as np
import nest_asyncio
from enum import Enum
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse

In [ ]:
# Присвоите экземпляр класса Fast API переменной "app".
# Вы будете взаимодействовать со своим api, используя этот экземпляр.
app = FastAPI(title='Deploying a ML Model with FastAPI')

# Перечислите доступные модели, используя Enum для удобства. Это полезно, когда параметры предопределены.
class Model(str, Enum):
    yolov3tiny = "yolov3-tiny"
    yolov3 = "yolov3"


# Используя @app.get("/"), вы разрешаете методу GET работать для конечной точки /.
@app.get("/")
def home():
    return "Поздравляю! Ваш API работает так, как надо. А теперь направляйтесь к http://localhost:8000/docs."


# Эта конечная точка обрабатывает всю логику, необходимую для обнаружения объекта.
# Для этого требуется желаемая модель и изображение, на котором выполняется обнаружение объекта.
@app.post("/predict") 
def prediction(model: Model, file: UploadFile = File(...)):

    # 1. ПРОВЕРКА ВХОДНОГО ФАЙЛА
    filename = file.filename
    fileExtension = filename.split(".")[-1] in ("jpg", "jpeg", "png")
    if not fileExtension:
        raise HTTPException(status_code=415, detail="Unsupported file provided.")
    
    # 2. ПРЕОБРАЗОВАНИЕ НЕОБРАБОТАННОГО ИЗОБРАЖЕНИЯ В изображение CV2 
    
    # Считывание изображения в виде потока байтов
    image_stream = io.BytesIO(file.file.read())
    
    # Запускаем поток с самого начала (нулевая позиция)
    image_stream.seek(0)
    
    # Запишем поток байтов в массив numpy
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    
    # Декодируем массив numpy в изображение CV2 
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    
    
    # 3. ЗАПУСТИТЬ МОДЕЛЬ ОБНАРУЖЕНИЯ ОБЪЕКТОВ
    
    # Запустим обнаружение объектов
    bbox, label, conf = cv.detect_common_objects(image, model=model)
    
    # Создадим изображение, включающее ограничительные рамки и метки
    output_image = draw_bbox(image, bbox, label, conf)
    
    # Сохраним его в папке на сервере
    cv2.imwrite(f'images_uploaded/{filename}', output_image)
    
    
    # 4. ПЕРЕДАДИМ ОТВЕТ ОБРАТНО КЛИЕНТУ
    
    # Откроем сохраненное изображение для чтения в двоичном формате
    file_image = open(f'images_uploaded/{filename}', mode="rb")
    
    # Возвращаем изображение в виде потока с указанием типа файла
    return StreamingResponse(file_image, media_type="image/jpeg")

Запустив следующую ячейку, вы запустите сервер!

Это приведет к блокировке текущего ноутбука (никакие ячейки / код не могут выполняться) до тех пор, пока вы вручную не перезапустите ядро. Вы можете сделать это, нажав на вкладку "Kernel", а затем на `Interrupt`. Вы также можете войти в командный режим Jupiter, нажав клавишу "ESC" и дважды нажав клавишу "I".

In [ ]:
# Позволяем запустить сервер в этой интерактивной среде
nest_asyncio.apply()

# Хост зависит от выбранной вами настройки (docker или virtualenv)
host = "0.0.0.0" if os.getenv("DOCKER-SETUP") else "127.0.0.1"

# Запускаем сервер!    
uvicorn.run(app, host=host, port=8000)

Сервер запущен! Направляйтесь к [http://localhost:8000 /](http://localhost:8000 /), чтобы увидеть его в действии.

**Попробуйте отправить изображение** и посмотрите, как ваш API может обнаруживать объекты и возвращать новое изображение, содержащее ограничительные рамки рядом с метками обнаруженных объектов. **Вы можете сделать это, посетив [http://localhost:8000/docs ](http://localhost:8000/docs ), чтобы открыть встроенный клиент fastAPI.**

При этом вы получите экран, который должен выглядеть так, как показано ниже, следуйте инструкциям далее:

Нажмите на верхнюю часть конечной точки "/predict", и станут видны дополнительные параметры:

![image.png](attachment:image.png)



Чтобы протестировать свой сервер, нажмите на кнопку **Try it out**.

![image.png](attachment:image.png)

Вы можете выбрать модель из поля **model** (если вы выберете полную модель YOLO, сервер будет заблокирован до тех пор, пока не будут загружены веса для этой модели) и **file**, который должен быть изображением, в котором вы хотите, чтобы сервер обнаруживал объекты.

**Отправьте изображение** из вашей локальной файловой системы, нажав кнопку **Choose File**, затем нажмите синюю кнопку **Execute**, чтобы отправить HTTP-запрос на сервер. После этого **прокрутите страницу вниз, и вы увидите ответ от сервера**. Довольно круто, правда?

![image.png](attachment:image.png)

**Попробуйте разные изображения!** Вы можете использовать те, которые предоставлены в этом задании, или свои собственные. Поскольку модель использует уровень достоверности по умолчанию, равный 0,5, обнаружение некоторых объектов может быть не всегда успешным.

Кроме того, попробуйте отправить файлы, не содержащие изображений, и посмотрите, как сервер отреагирует на это.

## Использование вашей модели в другом клиенте

Удивительно, что fast API позволяет вам взаимодействовать с вашим API через его встроенный клиент. Однако вы можете задаться вопросом, как вы можете взаимодействовать со своим API, используя обычный код, а не какой-то пользовательский интерфейс.

Для этого во второй части задания представлен минимальный клиент на Python. Для того, чтобы посмотреть как он работает **оставьте сервер работающим (не прерывайте ядро и не закрывайте это окно)** и откройте ноутбук `client.ipynb`. Для этого вы можете открыть вкладку Браузера файлов на боковой панели, расположенной слева от окна, и дважды щелкнуть по `client.ipynb`. Если вы не видите отдельную вкладку для каждого файла (что очень полезно для перемещения между ними), возможно, у вас включен режим "Simple Interface" (он же Одностраничный). Чтобы отключить его, вы можете перейти на вкладку "View" и отключить его.